In [1]:
import pandas as pd

In [2]:
games_reviews_items = pd.read_parquet('games_reviews_items.parquet')

In [8]:
games_reviews_items.tail(2)

,genres,title,price,item_id,developer,release_year,recommend,user_id,sentiment_analysis,posted_year,playtime_forever
113961,Free to play,Puzzle pirates,0.0,99910,Three rings,2011,True,WhenWeWereYoungggg,2,2012,360674
113962,Strategy,Puzzle pirates,0.0,99910,Three rings,2011,True,Jezza453,0,2015,360674


Debes crear las siguientes funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [51]:
def PlayTimeGenre(genero: str, data: pd.DataFrame):
    
    genre_data = data[data['genres'] == genero]

    
    year_playtime = genre_data.groupby('release_year')['playtime_forever'].sum()

    
    year_with_most_playtime = year_playtime.idxmax()

    return {"Año de lanzamiento con más minutos jugados para Género {}: {}".format(genero, year_with_most_playtime)}

In [52]:
PlayTimeGenre('Action', games_reviews_items)

{'Año de lanzamiento con más minutos jugados para Género Action: 2012'}

In [54]:
PlayTimeGenre('Strategy', games_reviews_items)

{'Año de lanzamiento con más minutos jugados para Género Strategy: 2010'}

In [55]:
PlayTimeGenre('Free to play', games_reviews_items)

{'Año de lanzamiento con más minutos jugados para Género Free to play: 2017'}

In [57]:
PlayTimeGenre('Adventure', games_reviews_items)

{'Año de lanzamiento con más minutos jugados para Género Adventure: 2011'}

In [56]:
PlayTimeGenre('Casual', games_reviews_items)

{'Año de lanzamiento con más minutos jugados para Género Casual: 2017'}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [46]:
def UserForGenre(genero: str, data: pd.DataFrame):
    
    genre_data = data[data['genres'] == genero]

    
    user_year_playtime = genre_data.groupby(['user_id', 'posted_year'])['playtime_forever'].sum().reset_index()

    
    user_with_most_playtime = user_year_playtime.groupby('user_id')['playtime_forever'].sum().idxmax()

    
    user_most_playtime_data = user_year_playtime[user_year_playtime['user_id'] == user_with_most_playtime]

    
    playtime_by_year = [{"Año": year, "Minutos": hours} for year, hours in user_most_playtime_data[['posted_year', 'playtime_forever']].values]

    
    result = {
        "Usuario con más minutos jugados para Género {}: {}".format(genero, user_with_most_playtime): playtime_by_year
    }

    return result

In [47]:
UserForGenre("Action", games_reviews_items)

{'Usuario con más minutos jugados para Género Action: BuffinMutton': [{'Año': 0,
   'Minutos': 30558990},
  {'Año': 2015, 'Minutos': 568974897}]}

In [58]:
UserForGenre("Strategy", games_reviews_items)

{'Usuario con más minutos jugados para Género Strategy: Slavishbear': [{'Año': 0,
   'Minutos': 36769818},
  {'Año': 2015, 'Minutos': 74161970}]}

In [48]:
UserForGenre("Free to play", games_reviews_items)

{'Usuario con más minutos jugados para Género Free to play: Killerbadboys': [{'Año': 2013,
   'Minutos': 30537228},
  {'Año': 2014, 'Minutos': 123876735}]}

In [59]:
UserForGenre("Adventure", games_reviews_items)

{'Usuario con más minutos jugados para Género Adventure: tarjla': [{'Año': 0,
   'Minutos': 5717355},
  {'Año': 2013, 'Minutos': 152423549},
  {'Año': 2014, 'Minutos': 106694311}]}

In [60]:
UserForGenre("Casual", games_reviews_items)

{'Usuario con más minutos jugados para Género Casual: frikcha': [{'Año': 2014,
   'Minutos': 26325264},
  {'Año': 2015, 'Minutos': 54364885}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [61]:
def UsersRecommend(año: int, data: pd.DataFrame):
    
    filtered_data = data[(data['posted_year'] == año) & (data['recommend'] == True) & (data['sentiment_analysis'].isin([1, 2]))]

    game_recommendations = filtered_data.groupby('title')['recommend'].count().reset_index()

    game_recommendations = game_recommendations.sort_values(by='recommend', ascending=False)

    top_3_games = game_recommendations.head(3)

    results = [{"Puesto {}: {}".format(i + 1, game)} for i, game in enumerate(top_3_games['title'])]

    return results

In [70]:
UsersRecommend(2011, games_reviews_items)

[{'Puesto 1: Terraria'}, {'Puesto 2: Portal 2'}, {"Puesto 3: Garry's mod"}]

In [69]:
UsersRecommend(2012, games_reviews_items)

[{'Puesto 1: Terraria'},
 {"Puesto 2: Garry's mod"},
 {'Puesto 3: Realm of the mad god'}]

In [62]:
UsersRecommend(2013, games_reviews_items)

[{"Puesto 1: Garry's mod"}, {'Puesto 2: Terraria'}, {'Puesto 3: Starbound'}]

In [63]:
UsersRecommend(2014, games_reviews_items)

[{'Puesto 1: Unturned'}, {'Puesto 2: Rust'}, {"Puesto 3: Garry's mod"}]

In [64]:
UsersRecommend(2015, games_reviews_items)

[{'Puesto 1: Counter-strike: global offensive'},
 {'Puesto 2: Unturned'},
 {'Puesto 3: Rust'}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [71]:
def UsersNotRecommend(año: int, data: pd.DataFrame):
    
    filtered_data = data[(data['posted_year'] == año) & (data['recommend'] == False) & (data['sentiment_analysis'] == 0)]

    game_not_recommendations = filtered_data.groupby('title')['recommend'].count().reset_index()

    game_not_recommendations = game_not_recommendations.sort_values(by='recommend', ascending=False)

    top_3_games = game_not_recommendations.head(3)

    results = [{"Puesto {}: {}".format(i + 1, game)} for i, game in enumerate(top_3_games['title'])]

    return results

In [72]:
UsersNotRecommend(2011, games_reviews_items)

[{'Puesto 1: And yet it moves'},
 {'Puesto 2: From dust'},
 {'Puesto 3: Men of war: vietnam'}]

In [73]:
UsersNotRecommend(2012, games_reviews_items)

[{'Puesto 1: Red faction®: armageddon™'},
 {'Puesto 2: Resident evil: operation raccoon city'},
 {"Puesto 3: The kings' crusade"}]

In [74]:
UsersNotRecommend(2013, games_reviews_items)

[{'Puesto 1: Rust'},
 {'Puesto 2: Nether: resurrected'},
 {'Puesto 3: No more room in hell'}]

In [75]:
UsersNotRecommend(2014, games_reviews_items)

[{'Puesto 1: Dayz'}, {'Puesto 2: Unturned'}, {'Puesto 3: Thehunter classic'}]

In [76]:
UsersNotRecommend(2015, games_reviews_items)

[{'Puesto 1: Dayz'}, {'Puesto 2: Rust'}, {'Puesto 3: Unturned'}]

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [77]:
def sentiment_analysis(año: int, data: pd.DataFrame):

    filtered_data = data[data['release_year'] == año]

    sentiment_counts = filtered_data['sentiment_analysis'].value_counts()

    results = {
        "Negative": sentiment_counts.get(0, 0),
        "Neutral": sentiment_counts.get(1, 0),
        "Positive": sentiment_counts.get(2, 0)
    }

    return results

In [80]:
sentiment_analysis(2011, games_reviews_items)

{'Negative': 1764, 'Neutral': 2082, 'Positive': 5634}

In [81]:
sentiment_analysis(2012, games_reviews_items)

{'Negative': 3635, 'Neutral': 3541, 'Positive': 9357}

In [83]:
sentiment_analysis(2013, games_reviews_items)

{'Negative': 4967, 'Neutral': 4321, 'Positive': 12137}

In [82]:
sentiment_analysis(2014, games_reviews_items)

{'Negative': 4225, 'Neutral': 3378, 'Positive': 9440}

In [84]:
sentiment_analysis(2015, games_reviews_items)

{'Negative': 4335, 'Neutral': 3814, 'Positive': 9848}